In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression
from math import sqrt
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std



plt.rc("axes.spines", top=False, right=False)


import wrangle
import split_scale
import evaluate
import features

Accuire Data

In [2]:
from env import host, user, password

In [3]:
#data for baseline model
df=wrangle.get_data_from_mysql()
df_engine=df

In [4]:
df_engine.head()

,transactiondate,id,bathrooms,bedrooms,lotsizesquarefeet,sqft,tax_value,region_id
0,2017-06-30,1772,2.0,4.0,6211.0,1604.0,498347.0,12447.0
1,2017-06-01,2028,3.0,3.0,9109.0,2384.0,549917.0,12447.0
2,2017-06-01,3273,2.0,3.0,7538.0,1574.0,235272.0,5534.0
3,2017-06-29,3429,2.0,2.0,111564.0,1619.0,340000.0,40227.0
4,2017-06-27,4505,3.0,2.0,10497.0,2408.0,2017254.0,12447.0


In [5]:
df_engine.drop(['transactiondate', 'id'], axis = 1).head()

,bathrooms,bedrooms,lotsizesquarefeet,sqft,tax_value,region_id
0,2.0,4.0,6211.0,1604.0,498347.0,12447.0
1,3.0,3.0,9109.0,2384.0,549917.0,12447.0
2,2.0,3.0,7538.0,1574.0,235272.0,5534.0
3,2.0,2.0,111564.0,1619.0,340000.0,40227.0
4,3.0,2.0,10497.0,2408.0,2017254.0,12447.0


In [6]:
#data for tax rate distribution
df_tax = wrangle.taxcounty()

In [7]:
df_tax.head()

,tax_value,fips,taxamount,tax_rate
0,498347.0,6037.0,6089.82,0.0122
1,549917.0,6037.0,6679.55,0.0121
2,235272.0,6037.0,3876.31,0.0165
3,340000.0,6037.0,4206.15,0.0124
4,2017254.0,6037.0,24353.42,0.0121


Clean Data

In [8]:
df_engine.head()

,transactiondate,id,bathrooms,bedrooms,lotsizesquarefeet,sqft,tax_value,region_id
0,2017-06-30,1772,2.0,4.0,6211.0,1604.0,498347.0,12447.0
1,2017-06-01,2028,3.0,3.0,9109.0,2384.0,549917.0,12447.0
2,2017-06-01,3273,2.0,3.0,7538.0,1574.0,235272.0,5534.0
3,2017-06-29,3429,2.0,2.0,111564.0,1619.0,340000.0,40227.0
4,2017-06-27,4505,3.0,2.0,10497.0,2408.0,2017254.0,12447.0


In [9]:
df_engine = df.filter(['bedrooms','bathrooms','sqft', 'lotsizesquarefeet', 'region_id','tax_value'], axis=1)

In [10]:
df_engine.head()

,bedrooms,bathrooms,sqft,lotsizesquarefeet,region_id,tax_value
0,4.0,2.0,1604.0,6211.0,12447.0,498347.0
1,3.0,3.0,2384.0,9109.0,12447.0,549917.0
2,3.0,2.0,1574.0,7538.0,5534.0,235272.0
3,2.0,2.0,1619.0,111564.0,40227.0,340000.0
4,2.0,3.0,2408.0,10497.0,12447.0,2017254.0


Split and Scale Data

In [11]:
train_engine, test_engine = split_scale.split_my_data(df_engine,train_ratio=.8,seed=123)

In [12]:
# impute a random column so we can test our feature engineering functions

scaler, train_scaled, test_scaled = \
    split_scale.standard_scaler(train_engine, test_engine)

In [13]:
X_train_engine = train_engine.drop(columns='tax_value')
y_train_engine = train_engine[['tax_value']]
X_test_engine = test_engine.drop(columns='tax_value')
y_test_engine = test_engine[['tax_value']]

Feature Engieering

In [16]:
train_engine['filler'] = -1
test_engine['filler'] = -1

train_scaled['filler'] = 1
test_scaled['filler'] = 1

In [17]:
train_scaled.head()

,bedrooms,bathrooms,sqft,lotsizesquarefeet,region_id,tax_value,filler
3396,-2.368441,-1.189873,-1.147011,-0.081050,-0.417016,-0.520056,1
9893,0.790348,-0.200811,0.012995,-0.043387,-0.164886,-0.262680,1
6069,-0.262581,-0.200811,-0.200004,-0.072112,-0.431133,-0.414941,1
14615,0.790348,0.788251,0.317875,-0.081616,-0.445783,0.427009,1
3172,-1.315511,-1.189873,-0.433884,-0.056350,0.177448,-0.048798,1


In [18]:
#Ktest
features.select_kbest_freg_unscaled(X_train_engine, y_train_engine, 3)

['bedrooms', 'bathrooms', 'sqft']

In [19]:
#Ktest
features.select_kbest_freg_scaled(X_train_engine, y_train_engine, 3)

['bedrooms', 'bathrooms', 'sqft']

In [20]:
#Backward elimination
#takes scaled dataframes (X and y) as input and returns selected features based on the ols backwards elimination method.
features.ols_backward_elimination(X_train_engine, y_train_engine)

['bedrooms', 'bathrooms', 'sqft']

In [29]:
features.optimal_features(X_train_engine, X_test_engine, y_train_engine, 3)

(Index(['bedrooms', 'bathrooms', 'sqft'], dtype='object'),
        bedrooms  bathrooms    sqft
 0           1.0        1.0   728.0
 1           4.0        2.0  1839.0
 2           3.0        2.0  1635.0
 3           4.0        3.0  2131.0
 4           2.0        1.0  1411.0
 5           3.0        1.0  1196.0
 6           4.0        2.0  1760.0
 7           6.0        4.0  6725.0
 8           2.0        2.0  1135.0
 9           2.0        1.0   713.0
 10          3.0        2.0  1272.0
 11          3.0        3.0  3188.0
 12          4.0        2.0  1931.0
 13          3.0        2.0  1917.0
 14          3.0        3.0  2220.0
 15          5.0        1.0  1477.0
 16          4.0        2.0  3119.0
 17          3.0        3.0  1738.0
 18          3.0        2.0  1726.0
 19          2.0        1.0   994.0
 20          3.0        2.0  2317.0
 21          4.0        3.0  2124.0
 22          3.0        1.0  1629.0
 23          4.0        3.5  2911.0
 24          2.0        3.0  2164.0
 25  